### Extracting POS Tags and Lemmatize the words - Telugu

**Notebook Author:** Nirupam Purushothama

1. Using the [library code](https://bitbucket.org/sivareddyg/telugu-part-of-speech-tagger/src/master/) of IIIT-LTRC research team to extract the POS Tags and Lemmas of words 
2. That library code is not updated to work with Python3.xx
3. I modified the code to work with Python3.xx and also work as a library for my use

I am checking in the code as part of my repository because my changes are temporary and will work with my code. For any latest updates you should refer to the IIIT-LTRC code for latest updates. The code here is just 

In [103]:
import os, sys, inspect
import importlib
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
grandparent_dir = os.path.dirname(parent_dir)
lib_dir = parent_dir + "/lib/lang_tools_te/bin"

if grandparent_dir not in sys.path:
    sys.path.insert(0, grandparent_dir)
    
if lib_dir not in sys.path:
    sys.path.insert(0, lib_dir)

In [104]:
import unitok_lib as unlib
importlib.reload(sys.modules['unitok_lib'])
import te_helpers as tehp
importlib.reload(sys.modules['te_helpers'])
import lemmatiser as lmt
importlib.reload(sys.modules['lemmatiser'])
import tag2vert as tv
importlib.reload(sys.modules['tag2vert'])

<module 'tag2vert' from '/home/ec2-user/learn/lib/lang_tools_te/bin/tag2vert.py'>

In [105]:
import json
import warnings
warnings.filterwarnings("ignore")

In [106]:
# Temp file names
words_tmp_file_name = "telugu_words.tmp"
tagger_op_file = "tagger_out.tmp"

# Tagger file path
lang_tools_path = "../lib/lang_tools_te"
tagger_file_path = lang_tools_path + "/bin/tnt"
models_path = lang_tools_path + "/models/telugu"

### The way these functions work are as follows:
1. Convert input sentences to tokens using unitok.py
2. Take these tokens and then get their POS tags using ./bin/tnt (unfortunately this has to be run like a script)
3. Then using this output call lemmatizer and get the lemmatized words
4. Then massage this data using tag2vert 

In [107]:
def write_to_tmp_file(te_list):
    with open(words_tmp_file_name, 'w') as file: 
        file.write("<doc>\n")
        
        for word in te_list:
            file.write(word+"\n")
        
        file.write("</doc>")
        
    return

def run_tagger_script():
    os.system(tagger_file_path + " -H -v0 "+ models_path + " " + words_tmp_file_name + 
              " | sed -e 's/\t\+/\t/g' > " + tagger_op_file)
    
    return

def read_tagger_output():
    texts_list = []
    
    with open(tagger_op_file, 'r') as file:
        file.readline()
        for line in file:
            texts_list.append(line.strip())
            
        texts_list.pop()
            
    return texts_list

In [109]:
te_string = "అసలు పిల్లలను కనక ముందు ఇక్కడ కనాలా అక్కడ కనాలా అని బోలెడంత సంఘర్షణ. ఇప్పుడు అమెరికా పౌరసత్వం తీసుకొనే అవకాశం వస్తే తీసుకోవాలా వద్దా అనేది మరో పెద్ద సంఘర్షణ."

In [110]:
te_list = unlib.tokenise(te_string, unlib.LANGUAGE_DATA['telugu']())
te_list = tehp.remove_non_words(te_list)
write_to_tmp_file(te_list)
run_tagger_script()
te_postags_list = read_tagger_output()
lmt.load_lemmatiser_default(lang_tools_path)

In [111]:
for item in te_postags_list:
    word_tag = item.split('\t')
    lemma = lmt.lemmatise_word(word_tag[0], word_tag[1])
    word, lemma_corr, pos_tag = tv.massage_lemma(word_tag[0], lemma, word_tag[1])
    print(word, lemma_corr, pos_tag)

అసలు అసలు NN
పిల్లలను పిల్ల NN
కనక కనకం CC
ముందు ముందు NST
ఇక్కడ ఇక్కడ NST
కనాలా కను VM
అక్కడ అక్కడ NST
కనాలా కను VM
అని అని UT
బోలెడంత బోలెడంత QF
సంఘర్షణ సంఘర్షణ NN
. None SYM.punc....
ఇప్పుడు ఇప్పుడు NST
అమెరికా అమెరికా NNP
పౌరసత్వం పౌరసత్వం NN
తీసుకొనే తీసుకో VM
అవకాశం అవకాశం NN
వస్తే వచ్చు VM
తీసుకోవాలా తీసుకో VM
వద్దా వద్దు VM
అనేది అను UT
మరో మరో QF
పెద్ద పెద్ద JJ
సంఘర్షణ సంఘర్షణ NN
. None SYM.punc....
